In [1]:
# Download the data from GCS
# !`wget 'https://storage.googleapis.com/movies_data/movies_metadata.csv'`

import ast
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import json
import pickle
import urllib

from sklearn.preprocessing import MultiLabelBinarizer

print(tf.__version__)

W0429 11:44:56.538949 4646532544 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


1.13.1


In [ ]:
EXPORT_PATH = '/your/path/to/export/the/model/versions/'

# Prepare the data

In [2]:
# inspect the data
data = pd.read_csv('movies_metadata.csv')
data.head()

/Users/hannes/.pyenv/versions/3.6.7/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
# data preprocessing 

descriptions = data['overview']
_genres = data['genres']

top_genres = ['Comedy', 'Thriller', 'Romance', 'Action', 'Horror', 'Crime', 'Documentary', 'Adventure', 'Science Fiction']
genres = []
for g in _genres:
    d = ast.literal_eval(g)
    k = []
    for i in d:
        if i['name'] in top_genres:
            k.append(i['name'])
    genres.append(k)

In [7]:
train_size = int(len(descriptions) * .8)

train_descriptions = descriptions[:train_size].astype('str')
train_genres = genres[:train_size]

test_descriptions = descriptions[train_size:].astype('str')
test_genres = genres[train_size:]

encoder = MultiLabelBinarizer()
encoder.fit_transform(train_genres)
train_encoded = encoder.transform(train_genres)
test_encoded = encoder.transform(test_genres)
num_classes = len(encoder.classes_)

# Print all possible genres and the labels for the first movie in our training dataset
print(encoder.classes_)
print(train_encoded[0])

['Action' 'Adventure' 'Comedy' 'Crime' 'Documentary' 'Horror' 'Romance'
 'Science Fiction' 'Thriller']
[0 0 1 0 0 0 0 0 0]


# Get an embedding layer from TensorFlow hub

In [14]:
description_embeddings = hub.text_embedding_column(
    "descriptions", module_spec="https://tfhub.dev/google/universal-sentence-encoder/1", trainable=False)

# Train your model, Deep NN Estimator

In [15]:
multi_label_head = tf.contrib.estimator.multi_label_head(
    num_classes,
    loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
)

features = {
  "descriptions": np.array(train_descriptions).astype(np.str)
}
labels = np.array(train_encoded).astype(np.int32)
train_input_fn = tf.estimator.inputs.numpy_input_fn(features, labels, shuffle=True, batch_size=32, num_epochs=10)
estimator = tf.estimator.DNNEstimator(
    head=multi_label_head,
    hidden_units=[64, 10],
    feature_columns=[description_embeddings],
)

estimator.train(input_fn=train_input_fn, steps=20)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using default config.


I0429 17:11:02.138752 4646532544 estimator.py:1739] Using default config.


W0429 17:11:02.143498 4646532544 estimator.py:1760] Using temporary folder as model directory: /var/folders/y7/r6t4nrpx4zg37s_bmtsz099h0000gn/T/tmpapibxlw2


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/y7/r6t4nrpx4zg37s_bmtsz099h0000gn/T/tmpapibxlw2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1370f5ac8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0429 17:11:02.149492 4646532544 estimator.py:201] Using config: {'_model_dir': '/var/folders/y7/r6t4nrpx4zg37s_bmtsz099h0000gn/T/tmpapibxlw2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1370f5ac8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


...

I0429 17:11:42.424072 4646532544 basic_session_run_hooks.py:249] loss = 0.69905424, step = 1


INFO:tensorflow:Saving checkpoints for 20 into /var/folders/y7/r6t4nrpx4zg37s_bmtsz099h0000gn/T/tmpapibxlw2/model.ckpt.


I0429 17:11:43.308335 4646532544 basic_session_run_hooks.py:594] Saving checkpoints for 20 into /var/folders/y7/r6t4nrpx4zg37s_bmtsz099h0000gn/T/tmpapibxlw2/model.ckpt.


INFO:tensorflow:Loss for final step: 0.62310237.


I0429 17:11:50.486005 4646532544 estimator.py:359] Loss for final step: 0.62310237.


# Let's evaluate the model

In [16]:
# Define our eval input_fn and run eval
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"descriptions": np.array(test_descriptions).astype(np.str)}, test_encoded.astype(np.int32), shuffle=False)
estimator.evaluate(input_fn=eval_input_fn)

samples = {"bond": """Quantum of Solace continues the adventures of James Bond after Casino Royale. Betrayed by Vesper, the woman he loved, 007 fights the urge to make his latest mission personal. Pursuing his determination to uncover the truth, 
                    Bond and M interrogate Mr. White, who reveals that the organization that blackmailed Vesper is far more complex and dangerous than anyone had imagined.""",
           "monty_python": """Monty Python and the Holy Grail loosely follows the legend of King Arthur. Arthur along with his squire, Patsy, recruits his
    Knights of the Round Table, including Sir Bedevere the Wise, Sir Lancelot the Brave, Sir Robin the
    Not-Quite-So-Brave-As-Sir-Lancelot and Sir Galahad the Pure. On the way Arthur battles the Black Knight who, despite having had
    all his limbs chopped off, insists he can still fight. They reach Camelot, but Arthur decides not to enter, as "it is a silly
    place".""",
            "sonnenallee": """A group of kids grow up on the short, wrong (east) side of the Sonnenallee in Berlin, right next to one of the few
    border crossings between East and West reserved for German citizens. The antics of these kids, their families, of the "West
    German" friends and relatives who come to visit, and of the East German border guards, all serve to illustrate the absurdity
    of everyday life on the Sonnenallee, and therefore throughout the former East Germany.""",
    "lifes_of_others": """A tragic love story set in East Berlin with the backdrop of an undercover Stasi controlled culture. Stasi captain
    Wieler is ordered to follow author Dreyman and plunges deeper and deeper into his life until he reaches the threshold of doubting
    the system."""}


for k, v in samples.items():
    predict_input_fn = tf.estimator.inputs.numpy_input_fn({"descriptions": np.array([v]).astype(np.str)}, None, shuffle=False)
    results = estimator.predict(input_fn=predict_input_fn)

    for movie_genres in results:
        print(f"+ {k}")
        top_2 = movie_genres['probabilities'].argsort()[-2:][::-1]
        for genre in top_2:
            text_genre = encoder.classes_[genre]
            print(text_genre + ': ' + str(round(movie_genres['probabilities'][genre] * 100, 2)) + '%')

INFO:tensorflow:Calling model_fn.


I0429 17:11:50.577944 4646532544 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0429 17:11:52.229658 4646532544 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Running local_init_op.


I0429 17:13:24.979109 4646532544 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0429 17:13:35.328118 4646532544 session_manager.py:493] Done running local_init_op.


+ lifes_of_others
Science Fiction: 49.73%
Thriller: 49.59%


# Let's export our model

In [17]:
def serving_input_receiver_fn():
  inputs = {
    "descriptions": tf.placeholder(tf.string, [None, 1]),
  }
  return tf.estimator.export.ServingInputReceiver(inputs, inputs)

# Save the model
saved_dir = EXPORT_PATH
estimator.export_savedmodel(
    saved_dir,
    serving_input_receiver_fn=serving_input_receiver_fn)

INFO:tensorflow:Calling model_fn.


I0429 17:13:56.876319 4646532544 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0429 17:13:58.352118 4646532544 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:No assets to write.


I0429 17:14:02.566149 4646532544 builder_impl.py:449] No assets to write.


INFO:tensorflow:SavedModel written to: /Users/hannes/tmp/movies/temp-b'1556583236'/saved_model.pb


I0429 17:14:08.344705 4646532544 builder_impl.py:414] SavedModel written to: /Users/hannes/tmp/movies/temp-b'1556583236'/saved_model.pb


b'/Users/hannes/tmp/movies/1556583236'

# Let's train our model again, more epochs this time

In [18]:
estimator.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Calling model_fn.


I0429 17:14:58.274204 4646532544 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0429 17:14:59.715361 4646532544 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

I0429 17:16:00.051098 4646532544 basic_session_run_hooks.py:247] loss = 0.33314142, step = 1921 (1.164 sec)


INFO:tensorflow:Saving checkpoints for 2020 into /var/folders/y7/r6t4nrpx4zg37s_bmtsz099h0000gn/T/tmpapibxlw2/model.ckpt.


I0429 17:16:01.366497 4646532544 basic_session_run_hooks.py:594] Saving checkpoints for 2020 into /var/folders/y7/r6t4nrpx4zg37s_bmtsz099h0000gn/T/tmpapibxlw2/model.ckpt.


INFO:tensorflow:Loss for final step: 0.29003173.


I0429 17:16:08.919802 4646532544 estimator.py:359] Loss for final step: 0.29003173.


In [19]:
# Define our eval input_fn and run eval
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"descriptions": np.array(test_descriptions).astype(np.str)}, test_encoded.astype(np.int32), shuffle=False)
estimator.evaluate(input_fn=eval_input_fn)

samples = {"bond": """Quantum of Solace continues the adventures of James Bond after Casino Royale. Betrayed by Vesper, the woman he loved, 007 fights the urge to make his latest mission personal. Pursuing his determination to uncover the truth, 
                    Bond and M interrogate Mr. White, who reveals that the organization that blackmailed Vesper is far more complex and dangerous than anyone had imagined.""",
           "monty_python": """Monty Python and the Holy Grail loosely follows the legend of King Arthur. Arthur along with his squire, Patsy, recruits his
    Knights of the Round Table, including Sir Bedevere the Wise, Sir Lancelot the Brave, Sir Robin the
    Not-Quite-So-Brave-As-Sir-Lancelot and Sir Galahad the Pure. On the way Arthur battles the Black Knight who, despite having had
    all his limbs chopped off, insists he can still fight. They reach Camelot, but Arthur decides not to enter, as "it is a silly
    place".""",
            "sonnenallee": """A group of kids grow up on the short, wrong (east) side of the Sonnenallee in Berlin, right next to one of the few
    border crossings between East and West reserved for German citizens. The antics of these kids, their families, of the "West
    German" friends and relatives who come to visit, and of the East German border guards, all serve to illustrate the absurdity
    of everyday life on the Sonnenallee, and therefore throughout the former East Germany.""",
    "lifes_of_others": """A tragic love story set in East Berlin with the backdrop of an undercover Stasi controlled culture. Stasi captain
    Wieler is ordered to follow author Dreyman and plunges deeper and deeper into his life until he reaches the threshold of doubting
    the system."""}


for k, v in samples.items():
    predict_input_fn = tf.estimator.inputs.numpy_input_fn({"descriptions": np.array([v]).astype(np.str)}, None, shuffle=False)
    results = estimator.predict(input_fn=predict_input_fn)

    for movie_genres in results:
        print(f"+ {k}")
        top_2 = movie_genres['probabilities'].argsort()[-2:][::-1]
        for genre in top_2:
            text_genre = encoder.classes_[genre]
            print(text_genre + ': ' + str(round(movie_genres['probabilities'][genre] * 100, 2)) + '%')

INFO:tensorflow:Calling model_fn.


I0429 17:17:43.074277 4646532544 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0429 17:17:44.553011 4646532544 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Running local_init_op.


I0429 17:19:11.038804 4646532544 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0429 17:19:21.335336 4646532544 session_manager.py:493] Done running local_init_op.


+ lifes_of_others
Thriller: 35.29%
Action: 23.83%


# Let's export a new model again

In [20]:
def serving_input_receiver_fn():
  inputs = {
    "descriptions": tf.placeholder(tf.string, [None, 1]),
  }
  return tf.estimator.export.ServingInputReceiver(inputs, inputs)

# Save the model
saved_dir = EXPORT_PATH
estimator.export_savedmodel(
    saved_dir,
    serving_input_receiver_fn=serving_input_receiver_fn)

INFO:tensorflow:Calling model_fn.


I0429 17:19:44.562821 4646532544 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0429 17:19:46.002534 4646532544 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:No assets to write.


I0429 17:19:49.895411 4646532544 builder_impl.py:449] No assets to write.


INFO:tensorflow:SavedModel written to: /Users/hannes/tmp/movies/temp-b'1556583584'/saved_model.pb


I0429 17:19:55.481378 4646532544 builder_impl.py:414] SavedModel written to: /Users/hannes/tmp/movies/temp-b'1556583584'/saved_model.pb


b'/Users/hannes/tmp/movies/1556583584'